In [80]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import pickle
from scipy.stats import kstest
from collections import Counter
from scipy.stats import sem 
import pingouin as pg

In [3]:
# INPUTS:

filepath = "F:/Two-Photon/Psilocybin Project/Evoked Cohort Mice/compiled_dicts"
z_thresh = 4

In [34]:
# Calculate the bandwidth of the cell using the half-max criterion, i.e. the continous range of responsive frequencies from the max response
# that show a response above 50% of the maximum response.  
# INPUTS:  Tuning array for the specified intensity (e.g. BF_column_1) in get_bandwidth_all_cells func.

def count_above_half_max(array):
    max_value = max(array)
    count = 0
    
    # Find the index of the maximum value in the array
    max_index = array.argmax()
    
    # Start from the index of the maximum value and iterate downwards
    index = max_index
    while index >= 0 and array[index] >= max_value / 2:
        count += 1
        index -= 1
    
    # Start from the index of the maximum value and iterate upwards
    index = max_index + 1  # Start from the next index
    while index < len(array) and array[index] >= max_value / 2:
        count += 1
        index += 1
    
    return count

In [5]:
# Dictionary to map filenames to variable names
file_variable_mapping = {
    'saline_pre_dict.pkl': 'saline_pre',
    'saline_post_dict.pkl': 'saline_post',
    'psilo_pre_dict.pkl': 'psilo_pre',
    'psilo_post_dict.pkl': 'psilo_post'
}

# Initialize empty dictionaries
saline_pre = {}
saline_post = {}
psilo_pre = {}
psilo_post = {}

# Iterate through files in megadict folder
for filename in os.listdir(filepath):
    if filename in file_variable_mapping:
        file_path = os.path.join(filepath, filename)
        with open(file_path, 'rb') as file:
            # Load pkl file and assign to respective dictionary variable
            globals()[file_variable_mapping[filename]] = pickle.load(file)

In [45]:
print(saline_pre['saline_1_186'][18]['peak_tuning'])

[[ 3.16538123e-01  1.77550443e+00  2.22869004e+00  5.66163589e-02]
 [ 2.00911175e-01  1.77368629e-01  2.40527420e-01  4.35925805e+00]
 [-2.52082877e-02  6.57310664e-02  2.33093291e-02  2.04954830e-01]
 [ 3.47114691e-01  4.41682848e-01  1.15507641e-01  3.05342290e-03]
 [ 7.07331762e-01  9.92378112e-01  7.26539551e-02  8.21607578e-01]
 [ 9.89843390e-01 -2.31348181e-02  9.78258737e-02  8.12818474e-02]
 [ 1.43558142e+00  6.04120067e-03  3.36815358e-01  1.14771081e+00]
 [ 1.22593969e-02  2.73537928e-01  6.21261304e-01  9.82298265e-02]
 [ 4.24616884e-02  4.82212575e-01  1.76020442e-02  1.15226519e-01]
 [ 9.06170423e-02  1.02153691e-01  8.15918672e-02  9.02214273e-03]
 [ 4.89330443e-02  5.67275012e-02 -6.88913318e-02  1.96792864e-01]
 [ 1.74754141e-02  1.15771675e-01  7.03608404e-02  7.25248793e-01]]


In [ ]:
'''
Pseudocode:

Add a 'bandwidth' key to each cell that will contain an array of 4 values. Fill with zeroes first.

For each cell, use the half_max function to compute the bandwidth at that sound level and store it in the array.




'''

In [ ]:



bandwidth_array = [0,0,0,0]

if saline_pre['saline_1_186'][12]['active'] == True:
    for level in range(No_sound_levels):
        level_array = saline_pre['saline_1_186'][4]['peak_tuning'][:,level]
        if any(value >= z_thresh for value in level_array):
            bandwidth_array[level] = count_above_half_max(level_array)
saline_pre['saline_1_186'][12]['bandwidth'] = bandwidth_array

In [55]:
def calculate_all_bandwidths(dict,No_sound_levels):

    sub_dict_keys = dict.keys()
    count = 0

    for sub_dict in sub_dict_keys:
        for cell in dict[sub_dict]:

            if dict[sub_dict][cell]['active'] == True:
                bandwidth_array = [0,0,0,0]
                count += 1

                for level in range(No_sound_levels):
                    level_array = dict[sub_dict][cell]['peak_tuning'][:,level]
                    if any(value >= z_thresh for value in level_array):
                        bandwidth_array[level] = count_above_half_max(level_array)
        
                dict[sub_dict][cell]['bandwidth'] = bandwidth_array
    print("The total number of cells in this cohort is", count)

        

In [56]:
No_sound_levels = 4
calculate_all_bandwidths(saline_pre,No_sound_levels)
calculate_all_bandwidths(saline_post,No_sound_levels)
calculate_all_bandwidths(psilo_pre,No_sound_levels)
calculate_all_bandwidths(psilo_post,No_sound_levels)

The total number of cells in this cohort is 1307
The total number of cells in this cohort is 1222
The total number of cells in this cohort is 1194
The total number of cells in this cohort is 1214


In [71]:
print(saline_pre['saline_1_186'][30]['bandwidth'])

[2, 1, 2, 2]


In [92]:
def dict_to_pd_bandwidth(dict,condition):
    rows = []

    for mouse_id, mouse_data in dict.items():

        base_mouse_id = mouse_id[-3:]

        for cell_id, cell_data in mouse_data.items():

            if cell_data['active'] == True:
                unique_cell_id = f"{mouse_id}_{condition}_{cell_id}"
                row = {
                    'original_mouse_id': mouse_id,
                    'mouse_id': base_mouse_id,
                    'cell': cell_id,
                    'condition': condition,
                    'unique_cell_id': unique_cell_id,
                    'bandwidth_35': cell_data['bandwidth'][0],
                    'bandwidth_50': cell_data['bandwidth'][1],
                    'bandwidth_65': cell_data['bandwidth'][2],
                    'bandwidth_80': cell_data['bandwidth'][3]
                }
                rows.append(row)
    df = pd.DataFrame(rows)

    return df

In [93]:
rows_pre_saline = dict_to_pd_bandwidth(saline_pre,'pre')
rows_post_saline = dict_to_pd_bandwidth(saline_post,'post')

rows_pre_psilo = dict_to_pd_bandwidth(psilo_pre,'pre')
rows_post_psilo = dict_to_pd_bandwidth(psilo_post,'post')

saline_data = pd.concat([rows_pre_saline,rows_post_saline])
psilo_data = pd.concat([rows_pre_psilo,rows_post_psilo])
print(saline_data.shape)
print(psilo_data.shape)



(2529, 9)
(2408, 9)


In [94]:
# First, prepare data in long format
def prepare_for_anova(df):
    long_df = pd.melt(
        df, 
        id_vars=['mouse_id', 'cell', 'condition'],
        value_vars=['bandwidth_35', 'bandwidth_50', 'bandwidth_65', 'bandwidth_80'],
        var_name='bandwidth_level', 
        value_name='bandwidth_value'
    )
    # Extract numeric bandwidth level
    long_df['bandwidth_level'] = long_df['bandwidth_level'].str.extract('(\d+)').astype(int)
    return long_df

# Run the split-plot ANOVA using pingouin
def run_split_plot_anova(long_df):
    # Mixed ANOVA with:
    # - condition as between-subjects factor
    # - bandwidth_level as within-subjects factor
    # - subject identifier as mouse_id:cell (each cell is a subject)
    aov = pg.mixed_anova(dv='bandwidth_value', 
                         within='bandwidth_level',
                         between='condition',
                         subject='cell', 
                         data=long_df)
    return aov

In [83]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

def run_nested_anova(long_df):
    # Nesting cells within condition and mice
    formula = 'bandwidth_value ~ C(condition) * C(bandwidth_level) + C(mouse_id)'
    model = ols(formula, data=long_df).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    return anova_table

In [86]:
prepared_saline = prepare_for_anova(saline_data)
prepared_psilo = prepare_for_anova(psilo_data)


In [103]:
# Create separate dataframes for each bandwidth level
bandwidth_levels = [35, 50, 65, 80]
results = {}

for bw in bandwidth_levels:
    # Filter data for this bandwidth level
    bw_data = saline_data[['mouse_id', 'unique_cell_id', 'condition', f'bandwidth_{bw}']]
    
    # Rename for clarity
    bw_data = bw_data.rename(columns={f'bandwidth_{bw}': 'bandwidth_value'})
    
    # Fit model for this bandwidth level
    model = smf.mixedlm(
        "bandwidth_value ~ condition", 
        bw_data, 
        groups=bw_data["mouse_id"]
    )
    
    try:
        # Store results if convergence successful
        results[bw] = model.fit()
        print(f"\nResults for bandwidth {bw}:")
        print(results[bw].summary().tables[1])  # Print only coefficients table
    except:
        print(f"\nConvergence failed for bandwidth {bw}")


Results for bandwidth 35:
                  Coef. Std.Err.      z  P>|z| [0.025 0.975]
Intercept         0.299    0.067  4.445  0.000  0.167  0.431
condition[T.pre]  0.052    0.024  2.199  0.028  0.006  0.098
Group Var         0.034    0.032                            

Results for bandwidth 50:
                  Coef. Std.Err.      z  P>|z| [0.025 0.975]
Intercept         0.371    0.067  5.548  0.000  0.240  0.502
condition[T.pre]  0.076    0.026  2.916  0.004  0.025  0.127
Group Var         0.033    0.028                            

Results for bandwidth 65:
                  Coef. Std.Err.      z  P>|z| [0.025 0.975]
Intercept         0.510    0.060  8.566  0.000  0.394  0.627
condition[T.pre]  0.074    0.032  2.326  0.020  0.012  0.136
Group Var         0.024    0.018                            

Results for bandwidth 80:
                  Coef. Std.Err.      z  P>|z| [0.025 0.975]
Intercept         0.545    0.066  8.226  0.000  0.415  0.675
condition[T.pre]  0.153    0.042  3.69

In [101]:
# Create separate dataframes for each bandwidth level
bandwidth_levels = [35, 50, 65, 80]
results = {}

for bw in bandwidth_levels:
    # Filter data for this bandwidth level
    bw_data = psilo_data[['mouse_id', 'unique_cell_id', 'condition', f'bandwidth_{bw}']]
    
    # Rename for clarity
    bw_data = bw_data.rename(columns={f'bandwidth_{bw}': 'bandwidth_value'})
    
    # Fit model for this bandwidth level
    model = smf.mixedlm(
        "bandwidth_value ~ condition", 
        bw_data, 
        groups=bw_data["mouse_id"]
    )
    
    try:
        # Store results if convergence successful
        results[bw] = model.fit()
        print(f"\nResults for bandwidth {bw}:")
        print(results[bw].summary().tables[1])  # Print only coefficients table
    except:
        print(f"\nConvergence failed for bandwidth {bw}")


Results for bandwidth 35:
                   Coef. Std.Err.       z  P>|z|  [0.025 0.975]
Intercept          0.347    0.058   5.985  0.000   0.233  0.460
condition[T.pre]  -0.035    0.024  -1.412  0.158  -0.082  0.013
Group Var          0.024    0.023                              

Results for bandwidth 50:
                  Coef. Std.Err.      z  P>|z|  [0.025 0.975]
Intercept         0.366    0.042  8.694  0.000   0.283  0.448
condition[T.pre]  0.009    0.027  0.319  0.750  -0.044  0.062
Group Var         0.011    0.011                             

Results for bandwidth 65:
                  Coef. Std.Err.      z  P>|z|  [0.025 0.975]
Intercept         0.453    0.052  8.728  0.000   0.351  0.555
condition[T.pre]  0.035    0.034  1.027  0.304  -0.032  0.101
Group Var         0.017    0.013                             

Results for bandwidth 80:
                  Coef. Std.Err.       z  P>|z| [0.025 0.975]
Intercept         0.438    0.039  11.359  0.000  0.362  0.514
condition[T.pre]

C:\Users\Conor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [104]:
import numpy as np
import pandas as pd
from statsmodels.stats.multitest import multipletests

# Your bandwidth levels
bandwidth_levels = [35, 50, 65, 80]
pvalues = []
coefs = []
stderr = []

# Use the correct parameter name: 'condition[T.pre]'
param_name = 'condition[T.pre]'

for bw in bandwidth_levels:
    try:
        # Extract p-value, coefficient, and standard error
        pvalues.append(results[bw].pvalues[param_name])
        coefs.append(results[bw].params[param_name])
        stderr.append(results[bw].bse[param_name])
    except KeyError:
        print(f"Parameter '{param_name}' not found for bandwidth {bw}")
        # Print available parameters to help debug
        print(f"Available parameters: {results[bw].params.index.tolist()}")
        pvalues.append(np.nan)
        coefs.append(np.nan)
        stderr.append(np.nan)

# Remove any NaN values before applying FDR correction
valid_indices = ~np.isnan(pvalues)
valid_pvals = [p for i, p in enumerate(pvalues) if valid_indices[i]]

if valid_pvals:
    # Apply FDR correction
    reject, pvals_corrected, _, _ = multipletests(valid_pvals, method='fdr_bh', alpha=0.05)
    
    # Put corrected values back in original order
    all_pvals_corrected = [np.nan] * len(pvalues)
    all_reject = [False] * len(pvalues)
    
    j = 0
    for i in range(len(pvalues)):
        if valid_indices[i]:
            all_pvals_corrected[i] = pvals_corrected[j]
            all_reject[i] = reject[j]
            j += 1
else:
    print("No valid p-values found for FDR correction")
    all_pvals_corrected = [np.nan] * len(pvalues)
    all_reject = [False] * len(pvalues)

# Create summary table
results_table = pd.DataFrame({
    'Bandwidth': bandwidth_levels,
    'Coefficient': coefs,
    'Std_Error': stderr,
    'Original_p': pvalues,
    'FDR_p': all_pvals_corrected,
    'Significant': all_reject
})

print("\nResults after FDR correction:")
print(results_table)

# Create a more detailed report
print("\nDetailed summary:")
for i, bw in enumerate(bandwidth_levels):
    if np.isnan(pvalues[i]):
        print(f"\nBandwidth {bw}: No valid results")
        continue
        
    sig_status = "significant" if all_reject[i] else "not significant"
    print(f"\nBandwidth {bw}:")
    print(f"  Effect of condition[T.pre]: {coefs[i]:.4f} ± {stderr[i]:.4f}")
    print(f"  Original p-value: {pvalues[i]:.4f}")
    print(f"  FDR-corrected p-value: {all_pvals_corrected[i]:.4f}")
    print(f"  Result: {sig_status} after FDR correction")


Results after FDR correction:
   Bandwidth  Coefficient  Std_Error  Original_p     FDR_p  Significant
0         35     0.051685   0.023502    0.027863  0.027863         True
1         50     0.075800   0.025994    0.003544  0.007089         True
2         65     0.073616   0.031647    0.020011  0.026681         True
3         80     0.153154   0.041504    0.000224  0.000897         True

Detailed summary:

Bandwidth 35:
  Effect of condition[T.pre]: 0.0517 ± 0.0235
  Original p-value: 0.0279
  FDR-corrected p-value: 0.0279
  Result: significant after FDR correction

Bandwidth 50:
  Effect of condition[T.pre]: 0.0758 ± 0.0260
  Original p-value: 0.0035
  FDR-corrected p-value: 0.0071
  Result: significant after FDR correction

Bandwidth 65:
  Effect of condition[T.pre]: 0.0736 ± 0.0316
  Original p-value: 0.0200
  FDR-corrected p-value: 0.0267
  Result: significant after FDR correction

Bandwidth 80:
  Effect of condition[T.pre]: 0.1532 ± 0.0415
  Original p-value: 0.0002
  FDR-correct